<h1>Chapter 12. Special Methods for Sequences</h1>

<h2><code>Vector</code> Attempt No.1: Compatibility with <code>Vector2d</code></h2>

Implementation of the first version of the `Vector` class (based on the code of the `Vector2d` class from chapter 11).

In [1]:
import math
import reprlib
from array import array


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        # Use reprlib.repr to get a limited length representation
        components = reprlib.repr(self._components)
        # Remove the prefix "array('d'" and closing bracket
        components = components[components.find('['): -1]
        return f"Vector({components})"

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes(ord[self.typecode]) + bytes(self._components)

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:].cast(typecode))
        return cls(memv)

In [2]:
Vector([3.1, 4.2])

Vector([3.1, 4.2])

In [3]:
Vector((3, 4, 5))

Vector([3.0, 4.0, 5.0])

In [4]:
Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])